# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
- Write queries that make use of various types of Joins
- Join tables using foreign keys

## CRM Schema

In almost all cases, rather then just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same Customer Relationship Management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [2]:
#Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [3]:
cur.execute("""SELECT * FROM sqlite_master""")
df = pd.DataFrame(cur.fetchall())
df

,0,1,2,3,4
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [4]:
cur.execute("""SELECT * FROM sqlite_master""").fetchall()

[('table',
  'orderdetails',
  'orderdetails',
  2,
  'CREATE TABLE `orderdetails` (`orderNumber`, `productCode`, `quantityOrdered`, `priceEach`, `orderLineNumber`)'),
 ('table',
  'payments',
  'payments',
  28,
  'CREATE TABLE `payments` (`customerNumber`, `checkNumber`, `paymentDate`, `amount`)'),
 ('table',
  'offices',
  'offices',
  32,
  'CREATE TABLE `offices` (`officeCode`, `city`, `phone`, `addressLine1`, `addressLine2`, `state`, `country`, `postalCode`, `territory`)'),
 ('table',
  'customers',
  'customers',
  33,
  'CREATE TABLE `customers` (`customerNumber`, `customerName`, `contactLastName`, `contactFirstName`, `phone`, `addressLine1`, `addressLine2`, `city`, `state`, `postalCode`, `country`, `salesRepEmployeeNumber`, `creditLimit`)'),
 ('table',
  'orders',
  'orders',
  38,
  'CREATE TABLE `orders` (`orderNumber`, `orderDate`, `requiredDate`, `shippedDate`, `status`, `comments`, `customerNumber`)'),
 ('table',
  'productlines',
  'productlines',
  46,
  'CREATE TABLE `

In [39]:
#Your code here
cur.execute("""SELECT *
               FROM employees
               JOIN offices
               USING (officeCode)
               WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [13]:
#Your code here
cur.execute("""SELECT city, COUNT(*)
               FROM employees
               JOIN offices
               USING (officeCode)
               GROUP BY city;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,city,COUNT(*)
0,Boston,2
1,London,2
2,NYC,2
3,Paris,5
4,San Francisco,6


## Write 3 Questions of your own and answer them

In [17]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
cur.execute("""SELECT htmlDescription, firstName, lastName
               FROM productlines
               LEFT JOIN employees
               GROUP BY lastName;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,htmlDescription,firstName,lastName
0,,Loui,Bondur
1,,Larry,Bott
2,,Anthony,Bow
3,,Pamela,Castillo
4,,Julie,Firrelli


In [35]:
# Your code here
cur.execute("""SELECT customerName, city, country
               FROM employees
               LEFT JOIN customers
               GROUP BY country
               ORDER BY city;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,customerName,city,country
0,Schuyler Imports,Amsterdam,Netherlands
1,Kelly's Gift Shop,Auckland,New Zealand
2,Precious Collectables,Bern,Switzerland
3,Diecast Collectables,Boston,USA
4,Scandinavian Gift Ideas,Bräcke,Sweden


In [33]:
# Your code here
cur.execute("""SELECT SUM(amount) AS dolla_billz, city, country
               FROM payments
               LEFT JOIN offices
               GROUP BY country
               ORDER BY dolla_billz DESC;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,dolla_billz,city,country
0,26561517.69,NYC,USA
1,8853839.23,Sydney,Australia
2,8853839.23,Paris,France
3,8853839.23,Tokyo,Japan
4,8853839.23,London,UK


In [41]:
# Your code here
cur.execute("""SELECT lastName, firstName, quantityOrdered
               FROM employees
               LEFT JOIN orderdetails
               GROUP BY quantityOrdered
               ORDER BY quantityOrdered DESC
               LIMIT 1;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,lastName,firstName,quantityOrdered
0,Gerard,Martin,97


## Level Up: Display the names of every individual product that each employee has sold

In [50]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees emp
               JOIN customers cus
               ON emp.employeeNumber = cus.salesRepEmployeeNumber
               JOIN orders ord
               USING(customerNumber)
               JOIN orderdetails deets
               USING(orderNumber)
               JOIN products prdct
               USING(productCode);""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
5,Leslie,Jennings,1968 Dodge Charger
6,Leslie,Jennings,1970 Plymouth Hemi Cuda
7,Leslie,Jennings,1969 Dodge Charger
8,Leslie,Jennings,1948 Porsche 356-A Roadster
9,Leslie,Jennings,1969 Dodge Super Bee


## Level Up: Display the Number of Products each employee has sold

In [60]:
cur.execute("""SELECT firstName, lastName, COUNT(productName) AS num_of_products
               FROM employees emp
               JOIN customers cus                   
               ON emp.employeeNumber = cus.salesRepEmployeeNumber
               JOIN orders ord
               USING(customerNumber)
               JOIN orderdetails deets 
               USING(orderNumber)                    
               JOIN products pdcts 
               USING(productCode)
               GROUP BY lastName
               ORDER BY num_of_products DESC;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,firstName,lastName,num_of_products
0,Gerard,Hernandez,396
1,Leslie,Jennings,331
2,Pamela,Castillo,272
3,Larry,Bott,236
4,Barry,Jones,220
5,George,Vanauf,211
6,Andy,Fixter,185
7,Peter,Marsh,185
8,Loui,Bondur,177
9,Steve,Patterson,152


## Display Total Products Sold by each employee

In [56]:
# Your code here
cur.execute("""SELECT COUNT(orderNumber) AS total_products_sold, firstName, lastName
               FROM orders
               JOIN employees
               GROUP BY lastName
               ORDER BY total_products_sold DESC;""")
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,total_products_sold,firstName,lastName
0,978,Steve,Patterson
1,652,Loui,Bondur
2,652,Julie,Firrelli
3,326,Larry,Bott
4,326,Anthony,Bow
5,326,Pamela,Castillo
6,326,Andy,Fixter
7,326,Martin,Gerard
8,326,Gerard,Hernandez
9,326,Leslie,Jennings


## Summary

Congrats! You now know how to use join statements, along with leveraging your foreign keys knowledge!